# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os

# Import API key
from api_keys import g_key

In [2]:
#!pip install gmaps
import gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv('../output_data/WeatherPy_Output.csv')
weather_data = weather_data.drop(columns = ["Unnamed: 0", "Unnamed: 0.1"])
weather_data.head(3)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hilo,90,US,1587244334,51,19.73,-155.09,27.00,4.10
1,Labuhan,76,ID,1587244334,85,-6.88,112.21,25.89,1.70
2,East London,0,ZA,1587244334,66,-33.02,27.91,21.79,7.84


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
max_humidity = weather_data.Humidity.max()
max_humidity

100

In [6]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [8]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Set criteria 
# Temp: 80F = 26.67C, 70F = 21.11, Wind Speed: 10mph = 4.47 meter/second
temp_criteria = (weather_data["Max Temp"] < 27) & (weather_data["Max Temp"] > 21)
windspeed_criteria = weather_data["Wind Speed"] < 4.5
cloudiness_criteria = weather_data["Cloudiness"] == 0

final_criteria = temp_criteria & windspeed_criteria & cloudiness_criteria 

ideal_weather_df = weather_data[final_criteria].reset_index()
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df.head(10)


,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,137,Bereket,0,TM,1587244351,37,39.24,55.52,21.16,3.67
1,308,Chiria,0,IN,1587244376,40,22.30,85.27,24.13,0.52
2,330,Atar,0,MR,1587244363,19,20.52,-13.05,23.88,3.72
3,447,Bayjī,0,IQ,1587244398,37,34.93,43.49,23.33,2.04


In [10]:
# Only 4 locations from the weather dataframe met the criteria

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_weather_df
hotel_df['Hotel Name'] = ""
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,137,Bereket,0,TM,1587244351,37,39.24,55.52,21.16,3.67,
1,308,Chiria,0,IN,1587244376,40,22.30,85.27,24.13,0.52,
2,330,Atar,0,MR,1587244363,19,20.52,-13.05,23.88,3.72,
3,447,Bayjī,0,IQ,1587244398,37,34.93,43.49,23.33,2.04,


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [13]:
# setting up parameters for each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    name_address = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing results... skipping.")


Missing results... skipping.
Missing results... skipping.
Missing results... skipping.


In [14]:
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,137,Bereket,0,TM,1587244351,37,39.24,55.52,21.16,3.67,
1,308,Chiria,0,IN,1587244376,40,22.30,85.27,24.13,0.52,
2,330,Atar,0,MR,1587244363,19,20.52,-13.05,23.88,3.72,Odar kanawal
3,447,Bayjī,0,IQ,1587244398,37,34.93,43.49,23.33,2.04,


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer on top of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25) )

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

# Add layer
fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)
# Display Map
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…